<a href="https://colab.research.google.com/github/AinzOwl/mysticai-colab/blob/main/mystic_ai_sd1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mystic.Ai SD 1.5 model deployment script**

Hello, I'm [Ainzoal](https://ainz.sbs) you can find me on mystic.ai discord server, I made this colab after suffering a lot to deploy my first model on v2 to point i quited but now v3 is out and with help of **Paul** i managed to deploy my first model, so i decided to give back and help other people who want to deploy on this super amazing platform.

this colab is really easy, and if you face any errors just ask, anyway i tried to document it and make it as clear as possible so here you go.

For SDXL [click here](https://github.com/AinzOwl/mysticai-colab/blob/main/mystic_ai_sdxl.ipynb)

# **downloading packages**

This was actually the error i faced i had missing packages, but worry not this will fix it just hit the download, also you dont need gpu for this so if it's enabled at top right u can just disable it to save on google resources.

In [ ]:
#@title Download Packages

!pip install pipeline-ai torch transformers diffusers accelerate xformers

# **setup api key**

When you run this it will ask you for your api key if you don't know where to find it it's [Here](https://mystic.ai/api-tokens) just type it and hit enter

In [ ]:
#@title Setup Mystic.Ai Api Key

api_key = input('Enter mystic.ai api key: ')
!pipeline cluster login catalyst-api {api_key} -u https://www.mystic.ai -a

# **import packages**
Yeah just run this one and that's it, nothing to change

In [ ]:
#@title Import packages
from pathlib import Path
from typing import List

import torch
from diffusers import StableDiffusionPipeline, AutoencoderKL

from pipeline import Pipeline, Variable, entity, pipe
from pipeline.cloud import compute_requirements, environments, pipelines
from pipeline.objects import File
from pipeline.objects.graph import InputField, InputSchema

# **inputs**

this is the configuration for the model inputs, you don't really need to change anything here unless you want to restrict some stuff or change some default values

to change default values it's easy just search for name of value you want to change and change its default, for example you want to set default height to 1024 then go to height and change 512 with 1024

In [ ]:
#@title Setup inputs

class ModelKwargs(InputSchema):
    height: int | None = InputField(
        title="Height",
        default=512,
        ge=64,
        le=1024,
        multiple_of=8,
    )

    width: int | None = InputField(
        title="Width",
        default=512,
        ge=64,
        le=1024,
        multiple_of=8,
    )

    num_inference_steps: int | None = InputField(
        title="Number of inference steps",
        default=25,
    )

    num_images_per_prompt: int | None = InputField(
        default=1, title="Number of images", ge=1, le=4
    )

    negative_prompt: str  | None = InputField(
        default='',
        title="Negative Prompt",
    )

    guidance_scale: float | None = InputField(
        default=7.5,
        title="Guidance scale",
        ge=0.0,
        le=20.0,
        # multiple_of=0.1,
    )

    allow_nsfw: bool | None = InputField(
        default=True,
        title="Allow NSFW",
    )

# **Setting up the model**

here you select your model and vae

for model just open the code of the block and replace prompthero/openjourney-v4 with your model id from huggingface.co

if you want to use a vae you can uncomment the `vae_model` and `vae=` by removing the `#` before them, please note some vae models on huggingface dont have config.json which will lead to an error.


In [ ]:
#@title Setup Model(open and replace the model id search for prompthero/openjourney-v4) to use a vae uncomment vae and replace "cumprod/yozora-vae" in vae model

@entity
class StableDiffusionModel:
    def __init__(self):
        ...

    @pipe(on_startup=True, run_once=True)
    def load(self):
        #vae_model = AutoencoderKL.from_pretrained("cumprod/yozora-vae")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_id = "prompthero/openjourney-v4"

        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            use_safetensors=True,
            #vae= vae_model,
        )
        self.pipe = self.pipe.to(device)

        def disabled_safety_checker(images, clip_input):
            if len(images.shape) == 4:
                num_images = images.shape[0]
                return images, [False] * num_images
            else:
                return images, False

        self.no_filter = disabled_safety_checker
        self.nsfw_filter = self.pipe.safety_checker

    @pipe
    def predict(self, prompt: str, kwargs: ModelKwargs) -> List[File]:
        defaults = kwargs.to_dict()
        del defaults["allow_nsfw"]

        if kwargs.allow_nsfw:
            self.pipe.safety_checker = self.no_filter

        images = self.pipe(prompt, **defaults).images

        output_images = []
        for i, image in enumerate(images):
            path = Path(f"/tmp/sd/image-{i}.jpg")
            path.parent.mkdir(parents=True, exist_ok=True)
            image.save(str(path))
            output_images.append(File(path=path, allow_out_of_context_creation=True))

        return output_images

# **Environment**

Env is the one holding the packages that your model uses, the ones selected are the ones needed to get this diffusers model to run and same env can be used for different models so you kinda really just need to run this one ones and you're good

In [ ]:
#@title setup mystic environment

with Pipeline() as builder:
    prompt = Variable(
        str,
        title="Prompt"
    )
    kwargs = Variable(
        ModelKwargs,
        title="Model kwargs",
    )

    model = StableDiffusionModel()

    model.load()

    output = model.predict(prompt, kwargs)

    builder.output(output)

my_pl = builder.get_pipeline()
try:
    environments.create_environment(
        "stable_diffusion_1_5",
        python_requirements=[
            "torch==2.0.1",
            "transformers==4.30.2",
            "diffusers==0.19.3",
            "accelerate==0.21.0",
            "xformers==0.0.21",
        ],
    )
except Exception:
    pass

# **Pushing the model**

pushing is easy you just replace `openjourney-v4` with what you named the model and thats it, if you used a different name for environement you can change it.

For vram it depends on model how much it need but dont overkill it just try to find the balance.

For accelerators check this page [Mystic Ai Docs](https://docs.mystic.ai/docs/gpus-and-accelerators) you'll find the id of each and pick one you want depending on price and vram

In [ ]:
#@title Push the model (open this to change required gpu type)
remote_pipeline = pipelines.upload_pipeline(
    my_pl,
    "openjourney-v4:latest",
    environment_id_or_name="stable_diffusion_1_5",
    required_gpu_vram_mb=20_000,
    accelerators=[
        compute_requirements.Accelerator.nvidia_a100,
    ],
)

print(remote_pipeline.id)